In [1]:
!pip install transformers

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.0 MB/s eta 0:00:00


In [3]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
!pip install openai

In [5]:
import torch

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [7]:
from datasets import load_dataset

In [8]:
import openai

In [9]:
import zipfile
import os


In [15]:
with zipfile.ZipFile("archive.zip", "r") as zip_ref:
    zip_ref.extractall("dataset")  # Extracts files to "dataset" folder


FileNotFoundError: [Errno 2] No such file or directory: 'archive.zip'

In [13]:
os.listdir("dataset")  # Shows all extracted files


FileNotFoundError: [Errno 2] No such file or directory: 'dataset'

In [14]:
pip install pandas

In [ ]:
import pandas as pd
import json

# Load JSON file
with open("dataset/intents.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Check the structure of the JSON data
print(type(data))  # Should print 'dict' or 'list'
print(data.keys())  # If it's a dict, check available keys

# Convert to DataFrame (depends on structure)
df = pd.DataFrame(data["intents"])  # Adjust the key as per your JSON structure

# Display first few rows
df.head()


In [ ]:
df.head()


In [ ]:
df.info()


In [ ]:
df.isnull().sum()


In [ ]:
df.columns


In [ ]:
print(df["patterns"].head())
print(df["responses"].head())


In [ ]:
df["tag"].value_counts()


In [ ]:
import matplotlib.pyplot as plt

df["tag"].value_counts().plot(kind="bar", figsize=(10, 5))
plt.xlabel("Intent")
plt.ylabel("Count")
plt.title("Intent Distribution")
plt.show()


In [ ]:
import re
def clean_text(context_set):
    context_set = re.sub(r'http\S+', '', context_set)  # Remove URLs
    context_set = re.sub(r'[^\w\s]', '', context_set)  # Remove punctuation
    context_set = context_set.lower()  # Convert to lowercase
    return context_set
df['context_set'] = df['context_set'].apply(clean_text)

In [ ]:
print(df.columns)


In [ ]:
print(df.columns)


In [ ]:
X = df["patterns"]
y = df["responses"]

print(X.head())  # Check first few questions
print(y.head())  # Check first few answers


In [ ]:
import pandas as pd

# Expand the "patterns" list into separate rows, duplicating "responses"
df_exploded = df.explode("patterns")

# Rename columns for clarity
df_exploded = df_exploded.rename(columns={"patterns": "question", "responses": "answers"})

# Define X (input) and y (output)
X = df_exploded["question"]  # Each row now has a single question
y = df_exploded["answers"]   # Each row still has a list of possible responses

# Check if the transformation worked
print(X.head())  # Should print individual questions, not lists
print(y.head())  # Should still contain response lists


In [ ]:
df.to_json('preprocessed_dataset.json', orient='records')  # Save as JSON


In [ ]:
!pip install transformers datasets


In [ ]:
from datasets import Dataset

# Convert Pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df_exploded[['question', 'answers']])


In [ ]:
from datasets import DatasetDict

# Split dataset into train & test (90% train, 10% test)
dataset = dataset.train_test_split(test_size=0.1)

# Check dataset structure
print(dataset)


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add a padding token because GPT-2 doesn't have one by default
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['question'], truncation=True, padding='max_length', max_length=512)

tokenized_dataset = dataset.map(preprocess_function, batched=True)


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

# Convert Pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df_exploded[['question', 'answers']])

# Split into train & test (90% train, 10% test)
dataset = dataset.train_test_split(test_size=0.1)

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Add padding token (GPT-2 doesn't have one by default)
tokenizer.pad_token = tokenizer.eos_token

# Tokenization function with labels
def preprocess_function(examples):
    # Convert list of answers into a single string
    answers = [" ".join(ans) if isinstance(ans, list) else ans for ans in examples["answers"]]

    inputs = tokenizer(examples["question"], truncation=True, padding='max_length', max_length=512)
    targets = tokenizer(answers, truncation=True, padding='max_length', max_length=512)

    inputs["labels"] = targets["input_ids"].copy()  # Copy labels correctly
    return inputs

# Tokenize dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    eval_strategy="epoch",  # Use correct evaluation strategy
    report_to="none",  # Disable Weights & Biases
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

trainer.train()


In [ ]:
model.save_pretrained('./my_chatbot_model')
tokenizer.save_pretrained('./my_chatbot_model')

In [ ]:
def calculate_cost(tokens_used):
    cost_per_token = 0.00002  # Example cost per token
    return tokens_used * cost_per_token

In [ ]:
from transformers import pipeline
sentiment_analyzer = pipeline('sentiment-analysis')
def analyze_sentiment(text):
    return sentiment_analyzer(text)